In [5]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [43]:
# define function to load train, test, and validation datasets
def load_dataset(path, number_of_classes):
    data = load_files(path)
    x = np.array(data['filenames'])
    y = np_utils.to_categorical(np.array(data['target']), number_of_classes)
    return x, y

In [7]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D

In [45]:
train_data_dir = "./data/train"
validation_data_dir = "./data/valid"
test_data_dir = "./data/test"
number_of_classes = 3
# load list of dog names
tissue_names = [item[20:-1] for item in sorted(glob("./data/train/*/"))]
img_width, img_height = 3008, 2000
target_width, target_height = 300, 300

In [9]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset(train_data_dir, number_of_classes)
valid_files, valid_targets = load_dataset(validation_data_dir, number_of_classes)
test_files, test_targets = load_dataset(test_data_dir, number_of_classes)

In [46]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(target_height, target_width))
    # convert PIL.Image.Image type to 3D tensor with shape (400, 400, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, target_height, target_width, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [47]:
print('There are %d total tissue categories.' % len(tissue_names))
print('There are %s total tissue images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training tissue images.' % len(train_files))
print('There are %d validation tissue images.' % len(valid_files))
print('There are %d test tissue images.'% len(test_files))

There are 3 total tissue categories.
There are 2750 total tissue images.

There are 2000 training tissue images.
There are 150 validation tissue images.
There are 600 test tissue images.


In [48]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [05:46<00:00,  1.73it/s]


In [49]:
### Build the network 
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(target_height, target_width, 3), name='conv_1'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_1'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='conv_2'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_2'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_3'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_3'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_4'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_4'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_5'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_5'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='conv_6'))
#model.add(Dropout(0.25))
#model.add(MaxPooling2D((2, 2), strides=(2, 2), name='pool_6'))

# Classification
model.add(GlobalMaxPooling2D())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.25))
#model.add(Dense(100, activation='relu'))
#model.add(Dropout(0.25))
model.add(Dense(3, activation='softmax'))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 300, 300, 64)      1792      
_________________________________________________________________
dropout_21 (Dropout)         (None, 300, 300, 64)      0         
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 150, 150, 64)      0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 150, 150, 64)      36928     
_________________________________________________________________
dropout_22 (Dropout)         (None, 150, 150, 64)      0         
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 75, 75, 64)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 75, 75, 128)       73856     
__________

In [50]:
# Compile model
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [51]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='saved_models/weights.cancer.hdf5', 
                               verbose=1, save_best_only=True)

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        #rotation_range=5,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

epochs = 30

In [52]:
# Fit model on training data
#model.fit(train_tensors, train_targets, 
#          validation_data=(valid_tensors, valid_targets),
#          epochs=epochs, batch_size=2, callbacks=[checkpointer], verbose=1)

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=10), 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, callbacks=[checkpointer], verbose=1)

Epoch 1/30
200/200 [==============================] - ETA: 12:39 - loss: 1.2003 - acc: 0.20 - ETA: 6:54 - loss: 0.9849 - acc: 0.5000 - ETA: 4:59 - loss: 0.9421 - acc: 0.533 - ETA: 4:01 - loss: 0.8948 - acc: 0.600 - ETA: 3:26 - loss: 1.0515 - acc: 0.580 - ETA: 3:03 - loss: 0.9705 - acc: 0.633 - ETA: 2:47 - loss: 0.9223 - acc: 0.657 - ETA: 2:37 - loss: 0.9179 - acc: 0.662 - ETA: 2:26 - loss: 0.9145 - acc: 0.655 - ETA: 2:18 - loss: 0.8894 - acc: 0.670 - ETA: 2:11 - loss: 0.8958 - acc: 0.663 - ETA: 2:05 - loss: 0.8868 - acc: 0.666 - ETA: 2:01 - loss: 0.8822 - acc: 0.669 - ETA: 1:58 - loss: 0.8466 - acc: 0.685 - ETA: 1:54 - loss: 0.8292 - acc: 0.693 - ETA: 1:52 - loss: 0.7917 - acc: 0.712 - ETA: 1:49 - loss: 0.7974 - acc: 0.711 - ETA: 1:46 - loss: 0.8551 - acc: 0.694 - ETA: 1:43 - loss: 0.8528 - acc: 0.694 - ETA: 1:42 - loss: 0.8602 - acc: 0.695 - ETA: 1:39 - loss: 0.8650 - acc: 0.685 - ETA: 1:38 - loss: 0.8717 - acc: 0.681 - ETA: 1:36 - loss: 0.8740 - acc: 0.682 - ETA: 1:34 - loss: 0.8786 

200/200 [==============================] - ETA: 1:12 - loss: 1.0019 - acc: 0.600 - ETA: 1:12 - loss: 0.8965 - acc: 0.650 - ETA: 1:12 - loss: 0.7446 - acc: 0.766 - ETA: 1:11 - loss: 0.7320 - acc: 0.775 - ETA: 1:11 - loss: 0.7669 - acc: 0.740 - ETA: 1:11 - loss: 0.8056 - acc: 0.716 - ETA: 1:11 - loss: 0.7945 - acc: 0.728 - ETA: 1:10 - loss: 0.7906 - acc: 0.725 - ETA: 1:10 - loss: 0.8127 - acc: 0.711 - ETA: 1:09 - loss: 0.8290 - acc: 0.700 - ETA: 1:09 - loss: 0.8178 - acc: 0.709 - ETA: 1:09 - loss: 0.8360 - acc: 0.691 - ETA: 1:08 - loss: 0.8481 - acc: 0.684 - ETA: 1:08 - loss: 0.8520 - acc: 0.678 - ETA: 1:08 - loss: 0.8419 - acc: 0.686 - ETA: 1:07 - loss: 0.8491 - acc: 0.681 - ETA: 1:07 - loss: 0.8416 - acc: 0.688 - ETA: 1:07 - loss: 0.8541 - acc: 0.677 - ETA: 1:06 - loss: 0.8529 - acc: 0.678 - ETA: 1:06 - loss: 0.8486 - acc: 0.680 - ETA: 1:05 - loss: 0.8480 - acc: 0.681 - ETA: 1:05 - loss: 0.8489 - acc: 0.681 - ETA: 1:05 - loss: 0.8470 - acc: 0.682 - ETA: 1:04 - loss: 0.8437 - acc: 0.683

200/200 [==============================] - ETA: 1:12 - loss: 0.6877 - acc: 0.800 - ETA: 1:12 - loss: 0.8959 - acc: 0.600 - ETA: 1:12 - loss: 0.9653 - acc: 0.566 - ETA: 1:11 - loss: 0.8490 - acc: 0.675 - ETA: 1:12 - loss: 0.8545 - acc: 0.680 - ETA: 1:14 - loss: 0.8630 - acc: 0.666 - ETA: 1:14 - loss: 0.8926 - acc: 0.642 - ETA: 1:13 - loss: 0.9109 - acc: 0.625 - ETA: 1:13 - loss: 0.8750 - acc: 0.655 - ETA: 1:12 - loss: 0.9105 - acc: 0.630 - ETA: 1:11 - loss: 0.8956 - acc: 0.645 - ETA: 1:11 - loss: 0.8984 - acc: 0.641 - ETA: 1:10 - loss: 0.8855 - acc: 0.646 - ETA: 1:10 - loss: 0.8727 - acc: 0.657 - ETA: 1:09 - loss: 0.8729 - acc: 0.653 - ETA: 1:09 - loss: 0.8806 - acc: 0.650 - ETA: 1:09 - loss: 0.8666 - acc: 0.658 - ETA: 1:08 - loss: 0.8711 - acc: 0.655 - ETA: 1:08 - loss: 0.8615 - acc: 0.663 - ETA: 1:07 - loss: 0.8607 - acc: 0.665 - ETA: 1:07 - loss: 0.8556 - acc: 0.666 - ETA: 1:07 - loss: 0.8445 - acc: 0.672 - ETA: 1:06 - loss: 0.8380 - acc: 0.678 - ETA: 1:06 - loss: 0.8349 - acc: 0.679

200/200 [==============================] - ETA: 1:12 - loss: 0.8489 - acc: 0.700 - ETA: 1:12 - loss: 0.9912 - acc: 0.550 - ETA: 1:11 - loss: 0.9615 - acc: 0.566 - ETA: 1:11 - loss: 0.8741 - acc: 0.650 - ETA: 1:11 - loss: 0.8512 - acc: 0.680 - ETA: 1:11 - loss: 0.8507 - acc: 0.683 - ETA: 1:10 - loss: 0.8637 - acc: 0.671 - ETA: 1:10 - loss: 0.8394 - acc: 0.687 - ETA: 1:09 - loss: 0.8629 - acc: 0.666 - ETA: 1:09 - loss: 0.8617 - acc: 0.660 - ETA: 1:09 - loss: 0.8470 - acc: 0.672 - ETA: 1:08 - loss: 0.8531 - acc: 0.666 - ETA: 1:08 - loss: 0.8557 - acc: 0.661 - ETA: 1:08 - loss: 0.8503 - acc: 0.664 - ETA: 1:08 - loss: 0.8314 - acc: 0.673 - ETA: 1:08 - loss: 0.8362 - acc: 0.668 - ETA: 1:07 - loss: 0.8515 - acc: 0.664 - ETA: 1:07 - loss: 0.8478 - acc: 0.666 - ETA: 1:06 - loss: 0.8425 - acc: 0.668 - ETA: 1:06 - loss: 0.8269 - acc: 0.680 - ETA: 1:06 - loss: 0.8128 - acc: 0.690 - ETA: 1:05 - loss: 0.8143 - acc: 0.690 - ETA: 1:05 - loss: 0.8030 - acc: 0.700 - ETA: 1:04 - loss: 0.8051 - acc: 0.695

200/200 [==============================] - ETA: 1:12 - loss: 1.1858 - acc: 0.500 - ETA: 1:12 - loss: 1.0936 - acc: 0.550 - ETA: 1:11 - loss: 1.0741 - acc: 0.533 - ETA: 1:11 - loss: 1.0184 - acc: 0.575 - ETA: 1:11 - loss: 1.0048 - acc: 0.580 - ETA: 1:11 - loss: 1.0329 - acc: 0.550 - ETA: 1:10 - loss: 1.0065 - acc: 0.571 - ETA: 1:10 - loss: 0.9859 - acc: 0.587 - ETA: 1:10 - loss: 0.9870 - acc: 0.588 - ETA: 1:09 - loss: 0.9738 - acc: 0.600 - ETA: 1:09 - loss: 0.9752 - acc: 0.590 - ETA: 1:09 - loss: 0.9583 - acc: 0.608 - ETA: 1:08 - loss: 0.9382 - acc: 0.630 - ETA: 1:08 - loss: 0.9354 - acc: 0.635 - ETA: 1:08 - loss: 0.9351 - acc: 0.633 - ETA: 1:07 - loss: 0.9393 - acc: 0.625 - ETA: 1:07 - loss: 0.9339 - acc: 0.629 - ETA: 1:06 - loss: 0.9191 - acc: 0.638 - ETA: 1:06 - loss: 0.9147 - acc: 0.642 - ETA: 1:06 - loss: 0.9126 - acc: 0.645 - ETA: 1:05 - loss: 0.9038 - acc: 0.652 - ETA: 1:05 - loss: 0.9017 - acc: 0.650 - ETA: 1:05 - loss: 0.9052 - acc: 0.643 - ETA: 1:04 - loss: 0.8934 - acc: 0.650

...

200/200 [==============================] - ETA: 1:13 - loss: 0.6017 - acc: 0.800 - ETA: 1:12 - loss: 0.5530 - acc: 0.850 - ETA: 1:12 - loss: 0.6184 - acc: 0.800 - ETA: 1:11 - loss: 0.6486 - acc: 0.775 - ETA: 1:11 - loss: 0.6443 - acc: 0.780 - ETA: 1:11 - loss: 0.6525 - acc: 0.766 - ETA: 1:10 - loss: 0.6752 - acc: 0.757 - ETA: 1:10 - loss: 0.6705 - acc: 0.750 - ETA: 1:10 - loss: 0.6714 - acc: 0.755 - ETA: 1:09 - loss: 0.6851 - acc: 0.750 - ETA: 1:09 - loss: 0.6848 - acc: 0.754 - ETA: 1:09 - loss: 0.7326 - acc: 0.733 - ETA: 1:08 - loss: 0.7234 - acc: 0.738 - ETA: 1:08 - loss: 0.7252 - acc: 0.735 - ETA: 1:07 - loss: 0.7055 - acc: 0.746 - ETA: 1:07 - loss: 0.7023 - acc: 0.750 - ETA: 1:07 - loss: 0.7074 - acc: 0.747 - ETA: 1:06 - loss: 0.7390 - acc: 0.733 - ETA: 1:06 - loss: 0.7294 - acc: 0.736 - ETA: 1:06 - loss: 0.7241 - acc: 0.740 - ETA: 1:05 - loss: 0.7231 - acc: 0.738 - ETA: 1:05 - loss: 0.7324 - acc: 0.731 - ETA: 1:05 - loss: 0.7294 - acc: 0.734 - ETA: 1:04 - loss: 0.7442 - acc: 0.725

200/200 [==============================] - ETA: 1:11 - loss: 0.4550 - acc: 0.900 - ETA: 1:12 - loss: 0.6729 - acc: 0.800 - ETA: 1:12 - loss: 0.7011 - acc: 0.766 - ETA: 1:11 - loss: 0.7355 - acc: 0.750 - ETA: 1:11 - loss: 0.7829 - acc: 0.720 - ETA: 1:11 - loss: 0.8352 - acc: 0.683 - ETA: 1:11 - loss: 0.8232 - acc: 0.700 - ETA: 1:11 - loss: 0.7744 - acc: 0.725 - ETA: 1:10 - loss: 0.8028 - acc: 0.700 - ETA: 1:10 - loss: 0.8332 - acc: 0.680 - ETA: 1:10 - loss: 0.8148 - acc: 0.690 - ETA: 1:09 - loss: 0.8032 - acc: 0.700 - ETA: 1:09 - loss: 0.8005 - acc: 0.700 - ETA: 1:09 - loss: 0.8197 - acc: 0.685 - ETA: 1:09 - loss: 0.8316 - acc: 0.680 - ETA: 1:08 - loss: 0.8385 - acc: 0.675 - ETA: 1:08 - loss: 0.8408 - acc: 0.670 - ETA: 1:08 - loss: 0.8341 - acc: 0.677 - ETA: 1:07 - loss: 0.8393 - acc: 0.673 - ETA: 1:07 - loss: 0.8382 - acc: 0.670 - ETA: 1:07 - loss: 0.8403 - acc: 0.666 - ETA: 1:06 - loss: 0.8286 - acc: 0.677 - ETA: 1:06 - loss: 0.8336 - acc: 0.673 - ETA: 1:06 - loss: 0.8359 - acc: 0.670

200/200 [==============================] - ETA: 1:23 - loss: 0.9816 - acc: 0.500 - ETA: 1:21 - loss: 0.8602 - acc: 0.600 - ETA: 1:22 - loss: 0.8024 - acc: 0.666 - ETA: 1:23 - loss: 0.7354 - acc: 0.725 - ETA: 1:20 - loss: 0.7251 - acc: 0.740 - ETA: 1:18 - loss: 0.7777 - acc: 0.700 - ETA: 1:17 - loss: 0.7773 - acc: 0.700 - ETA: 1:18 - loss: 0.8129 - acc: 0.675 - ETA: 1:17 - loss: 0.7734 - acc: 0.700 - ETA: 1:15 - loss: 0.7568 - acc: 0.710 - ETA: 1:14 - loss: 0.7351 - acc: 0.727 - ETA: 1:14 - loss: 0.7650 - acc: 0.708 - ETA: 1:14 - loss: 0.7556 - acc: 0.715 - ETA: 1:13 - loss: 0.7501 - acc: 0.721 - ETA: 1:12 - loss: 0.7526 - acc: 0.720 - ETA: 1:12 - loss: 0.7592 - acc: 0.712 - ETA: 1:11 - loss: 0.7771 - acc: 0.700 - ETA: 1:10 - loss: 0.7968 - acc: 0.688 - ETA: 1:10 - loss: 0.7984 - acc: 0.689 - ETA: 1:10 - loss: 0.7914 - acc: 0.695 - ETA: 1:09 - loss: 0.7754 - acc: 0.704 - ETA: 1:09 - loss: 0.7679 - acc: 0.709 - ETA: 1:08 - loss: 0.7715 - acc: 0.708 - ETA: 1:07 - loss: 0.7653 - acc: 0.712

200/200 [==============================] - ETA: 1:12 - loss: 0.6100 - acc: 0.800 - ETA: 1:12 - loss: 0.8399 - acc: 0.650 - ETA: 1:12 - loss: 0.7685 - acc: 0.700 - ETA: 1:12 - loss: 0.8254 - acc: 0.650 - ETA: 1:11 - loss: 0.7838 - acc: 0.680 - ETA: 1:11 - loss: 0.7942 - acc: 0.700 - ETA: 1:10 - loss: 0.8303 - acc: 0.685 - ETA: 1:10 - loss: 0.8665 - acc: 0.662 - ETA: 1:10 - loss: 0.8549 - acc: 0.666 - ETA: 1:09 - loss: 0.8220 - acc: 0.690 - ETA: 1:09 - loss: 0.8512 - acc: 0.663 - ETA: 1:08 - loss: 0.8574 - acc: 0.658 - ETA: 1:08 - loss: 0.8528 - acc: 0.661 - ETA: 1:08 - loss: 0.8348 - acc: 0.671 - ETA: 1:08 - loss: 0.8487 - acc: 0.660 - ETA: 1:07 - loss: 0.8588 - acc: 0.650 - ETA: 1:07 - loss: 0.8620 - acc: 0.652 - ETA: 1:06 - loss: 0.8573 - acc: 0.655 - ETA: 1:06 - loss: 0.8536 - acc: 0.657 - ETA: 1:06 - loss: 0.8623 - acc: 0.650 - ETA: 1:05 - loss: 0.8699 - acc: 0.642 - ETA: 1:05 - loss: 0.8779 - acc: 0.640 - ETA: 1:05 - loss: 0.8627 - acc: 0.652 - ETA: 1:04 - loss: 0.8527 - acc: 0.658

200/200 [==============================] - ETA: 1:12 - loss: 1.1040 - acc: 0.500 - ETA: 1:12 - loss: 1.1220 - acc: 0.500 - ETA: 1:12 - loss: 0.9723 - acc: 0.600 - ETA: 1:11 - loss: 0.8915 - acc: 0.650 - ETA: 1:11 - loss: 0.9128 - acc: 0.640 - ETA: 1:11 - loss: 0.8559 - acc: 0.683 - ETA: 1:10 - loss: 0.8787 - acc: 0.671 - ETA: 1:10 - loss: 0.8059 - acc: 0.712 - ETA: 1:09 - loss: 0.7782 - acc: 0.722 - ETA: 1:09 - loss: 0.7928 - acc: 0.710 - ETA: 1:09 - loss: 0.8240 - acc: 0.690 - ETA: 1:08 - loss: 0.8238 - acc: 0.691 - ETA: 1:08 - loss: 0.8382 - acc: 0.676 - ETA: 1:08 - loss: 0.8324 - acc: 0.678 - ETA: 1:07 - loss: 0.8452 - acc: 0.673 - ETA: 1:07 - loss: 0.8251 - acc: 0.687 - ETA: 1:07 - loss: 0.8080 - acc: 0.700 - ETA: 1:06 - loss: 0.8075 - acc: 0.700 - ETA: 1:06 - loss: 0.8321 - acc: 0.684 - ETA: 1:06 - loss: 0.8236 - acc: 0.690 - ETA: 1:05 - loss: 0.8212 - acc: 0.690 - ETA: 1:05 - loss: 0.8118 - acc: 0.695 - ETA: 1:04 - loss: 0.8007 - acc: 0.700 - ETA: 1:04 - loss: 0.7968 - acc: 0.704

In [53]:
model.load_weights('saved_models/weights.cancer.hdf5')

In [54]:
# get index of predicted clas for each image in test set
class_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(class_predictions)==np.argmax(test_targets, axis=1))/len(class_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 65.5000%


In [55]:
import csv
class_probabilities = [model.predict(np.expand_dims(tensor, axis=0)) for tensor in test_tensors]

In [56]:
import csv
file_index = 0
with open('results.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    csv_writer.writerow(['Id'] + ['task_1'] + ['task_2'])
    for result in class_probabilities:
        csv_writer.writerow([test_files[file_index]] + [result[0,0]] + [result[0,2]])
        file_index = file_index + 1